# 3.2 Aprendizaje supervisado: Algoritmos basados en vecindad (neighborhood-based algorithms)

Profesor: Juan Ramón Rico (<juanramonrico@ua.es>)

## Resumen
---

Se presentarán los principios básicos de los algoritmos basados en vecindad, concretamente el conocido como el $k$ vecino más cercano.

In [23]:
import numpy as np
import pandas as pd

# Técnicas basadas en vecindad

- Estas técnicas se basan en funciones distancias que permiten comparar las muestras de entrada por pares para obtener una medida de similitud o parecido.
- Uno de los algoritmos más utilizados es el llamado k-NN (k Nearest Neighbors) que se basa en los valores objetivo de los vecinos más próximos para:
   - estimar la función de densidad y calcular la probabilidad a posteriori, $P(ω_i |x) = k_i /k$, de que una muestra $x$ pertenezca a la clase $ω_i$ ($k_i$ es el número de vecinos de la clase i) en problema de clasificación;
   - estimar el valor esperado en problemas de regresión, habitualmente calculando la media o mediana de los valores objetivo de los vecinos más próximos.

In [24]:
## Datos de prueba

path = 'https://www.dlsi.ua.es/~juanra/UA/datasets'
path_tennis = f'{path}/tennis-en.csv'
path_covid19 = f'{path}/covid19-en.csv'

## Función distancia

En este caso vamos a usar la más habitual, la distancia `Euclídea` o `L2`.

$$d_{L2}(\overrightarrow{x},\overrightarrow{y})=\sqrt{\sum_i(x_i-y_i)^2}$$

siendo $x$ e $y$ vectores de números.

In [25]:
def d_L2(x,y):
  # Calcula la distancia Euclídea o L2 entre dos vectores de números
  return np.sqrt(np.sum((x-y)**2))

# Caso de prueba
x = np.array([1,2])
y = np.array([2,3])

# Resultado 1.41
d_L2(x,y)

1.4142135623730951

## Ejemplo 1: Calculo del vecino más cercano

Vamos a implementar una función cuyos parámetros son una serie de items y un conjunto para que calcule el vecino más cercano de cada item al conjunto dado.

In [26]:
def nearest_neighbors(items, X):
  # Encuentra el vecino más cercano de una serie de items a un conjunto X
  neighbors = []

  for i, item1 in enumerate(items):
    min_distance = float('inf')
    nearest_neighbor = None

    for j, item2 in enumerate(X):
      distance = d_L2(item1, item2)
      if distance < min_distance:
        min_distance = distance
        nearest_neighbor = item2

    neighbors.append([item1, nearest_neighbor])

  return neighbors

# Ejemplo de uso
m_points = np.array([ [1, 2], [1, 1], [8, 8] ])
X_points = np.array([ [1, 2], [3, 4], [5, 6], [7, 8] ])
result = nearest_neighbors(m_points, X_points)

result

[[array([1, 2]), array([1, 2])],
 [array([1, 1]), array([1, 2])],
 [array([8, 8]), array([7, 8])]]

Resultado

Cada línea corresponde al item de entrada y su vecino más cercano.

```
[[array([1, 2]), array([1, 2])],
 [array([1, 1]), array([1, 2])],
 [array([8, 8]), array([7, 8])]]
```

## Ejemplo 2: Clasificación mediante vecinos más cercanos

Implementa una función en Python cuyos parámetros sean un conjunto de muestras sus respectivas clases (etiquetas) y una muestra para clasificar utilizando el algoritmo de vecinos más cercanos. En este caso, puedes utilizar la moda (la etiqueta más repetida) de los vecinos más cercanos como la etiqueta asignada a la nueva muestra.

In [46]:
import statistics

from scipy.spatial import distance
import statistics

def k_nearest_neighbors(training_vector, training_labels, test_vector, k, debug=False):
  distances = []

  # Calcula la distancia euclidiana entre el test_vector y cada punto en training_vector
  for i, point in enumerate(training_vector):
      dist = distance.euclidean(point, test_vector)
      distances.append((dist, training_labels[i]))

    
  #ESTO ES LO QUE HEMOS AÑADIDO  
  # Ordena la lista de distancias en orden ascendente y toma las primeras k distancias
  sorted_distances = sorted(distances)[:k]

  if debug:
    print(f'test_vector: {test_vector} distances: {sorted_distances}')

  # Extrae las etiquetas de las k distancias más cortas
  nearest_labels = [label for _, label in sorted_distances]
  print(f"Etiquetas más cercanas {nearest_labels}")

  # Calcula la moda (el valor más común) entre las etiquetas de los vecinos más cercanos
  most_common = statistics.mode(nearest_labels)

  return most_common

# Ejemplo de uso
training_vector = np.array([ [1, 2], [3, 4], [5, 6], [7, 8] ])
training_labels = ['A', 'B', 'A', 'B']
test_vector = np.array([4, 5])
k = 3

result = k_nearest_neighbors(training_vector, training_labels, test_vector, k, debug=True)
print(result)

test_vector: [4 5] distances: [(1.4142135623730951, 'A'), (1.4142135623730951, 'B'), (4.242640687119285, 'A')]
Etiquetas más cercanas ['A', 'B', 'A']
A


# Ejercicio 1: Tenis

## Implementación básica de los algoritmos

Utilizando el conjunto de `Tenis` y las funciones necesarias de los ejemplos anteriores clasificad las siguientes muestras para una `k=3`:

In [71]:
test_examples = pd.DataFrame([
    ['sunny',  'high', 'high', 'no'],
    ['rain', 'high', 'high', 'yes'],
    ['overcast',  'high', 'high', 'no']
  ],
  columns=['weather', 'temperature', 'humidity', 'wind']
)
test_examples

,weather,temperature,humidity,wind
0,sunny,high,high,no
1,rain,high,high,yes
2,overcast,high,high,no


In [72]:
# Cargar los datos de tenis
data = pd.read_csv(path_tennis)
data

,weather,temperature,humidity,wind,play
0,sunny,high,high,no,no
1,sunny,high,high,yes,no
2,overcast,high,high,no,yes
3,rain,medium,high,no,yes
4,rain,low,normal,no,yes
5,rain,low,normal,yes,no
6,overcast,low,normal,yes,yes
7,sunny,medium,high,no,no
8,sunny,low,normal,no,yes
9,rain,medium,normal,no,yes


Para usar la distancia de `euclídea` necesitamos trasformar las etiquetas de las clases en un formato numérico. Para ello podemos usar la conversión llamada `one-hot` que consiste en transformar cada columna que contenga una clase en tantas columnas como sus categorías.

El paquete `Pandas` contiene una función que realiza esta transformación, `pd.get_dummies()` y que podemos usar, o bien, se puede transformar implementando una nueva función que lo haga.

In [73]:
# TODO: Transformación de los datos (data) a un formato one-hot
data_encoded = pd.get_dummies(data)
data_encoded

,weather_overcast,weather_rain,weather_sunny,temperature_high,temperature_low,temperature_medium,humidity_high,humidity_normal,wind_no,wind_yes,play_no,play_yes
0,0,0,1,1,0,0,1,0,1,0,1,0
1,0,0,1,1,0,0,1,0,0,1,1,0
2,1,0,0,1,0,0,1,0,1,0,0,1
3,0,1,0,0,0,1,1,0,1,0,0,1
4,0,1,0,0,1,0,0,1,1,0,0,1
5,0,1,0,0,1,0,0,1,0,1,1,0
6,1,0,0,0,1,0,0,1,0,1,0,1
7,0,0,1,0,0,1,1,0,1,0,1,0
8,0,0,1,0,1,0,0,1,1,0,0,1
9,0,1,0,0,0,1,0,1,1,0,0,1


Vamos a usar la misma transformación anterior pero en esta ocasión a los datos de las nuevas muestras, variable `test_examples`.\

In [74]:
# Transformación de los datos de test (test_examples) a un formato one-hot
test_examples_encoded = pd.get_dummies(test_examples)

A continuación buscaremos los k vecinos más cercanos a las nuevas muestras del conjunto de entrenamiento y presentaremos la predicción de su clase.

In [75]:
# Ejemplo de clasificación para tenis con k=3 de las muestras de test (test_data) respecto de las de entrenamiento (data)
# Preparamos los datos de entrenamiento
X_train = data_encoded.drop('play_yes', axis=1)
training_vector = X_train.values
training_labels = data_encoded['play_yes'].values



# Agregamos las columnas faltantes al DataFrame de test_examples_encoded que están en X_train pero no en test_examples_encoded
missing_cols = set(X_train.columns) - set(test_examples_encoded.columns)
for c in missing_cols:
    test_examples_encoded[c] = 0

# Asegúrese de que el orden de las columnas coincida
test_examples_encoded = test_examples_encoded[X_train.columns]

# Convertimos el dataframe procesado en un array de numpy para usarlo en la clasificación
test_vectors = test_examples_encoded.values
print(test_vectors)

k = 3
for index, test_vec in test_examples_encoded.iterrows():
    test_vec = test_vec.values
    result = k_nearest_neighbors(training_vector, training_labels, test_vec, k)
    print(f"La muestra {index} pertenece a la clase: {'Yes' if result == 1 else 'No'}")

[[0 0 1 1 0 0 1 0 1 0 0]
 [0 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 1 0 0 1 0 1 0 0]]
Etiquetas más cercanas [0, 1, 0]
La muestra 0 pertenece a la clase: No
Etiquetas más cercanas [0, 0, 1]
La muestra 1 pertenece a la clase: No
Etiquetas más cercanas [1, 1, 0]
La muestra 2 pertenece a la clase: Yes


## Implementación con scikit-learn

El paquete `scikit-learn` contiene diferentes funciones basadas en algoritmos de vecindad. Para clasificación existe la función `sklearn.neighbors.KNeighborsClassifier()` que podemos usar para entrenar en primer lugar y predecir nuevos valores a continuación.

In [70]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3) # La distancia euclídea se usa por defecto


X_train = data_encoded.drop(['play_yes','play_no'], axis=1)
y_train = data_encoded['play_yes'].values


# Clasificación de 'test_data' respecto a 'data' usando sklearn
model.fit(X_train, y_train)

# Transformamos los datos de test (test_examples) a un formato one-hot
test_examples_encoded = pd.get_dummies(test_examples)

# Agregamos las columnas faltantes al DataFrame de test_examples_encoded que están en X_train pero no en test_examples_encoded
missing_cols = set(X_train.columns) - set(test_examples_encoded.columns)
for c in missing_cols:
    test_examples_encoded[c] = 0

# Aseguramos que el orden de las columnas en los datos de prueba coincida con los datos de entrenamiento
test_examples_encoded = test_examples_encoded.reindex(columns=X_train.columns, fill_value=0)

# Realizamos la clasificación de las muestras de prueba
predictions = model.predict(test_examples_encoded)

print(predictions)
'''
Resultado:

array(['no', 'no', 'yes'], dtype=object)
'''

KeyError: "['play_yes' 'play_no'] not found in axis"

# Ejercicio 2: COVID-19

## Implementación básica de los algoritmos

Utilizando el conjunto de `COVID-19` y las funciones necesarias de los ejemplos anteriores clasificad las siguientes muestras para una `k=3`:

In [57]:
test_examples = pd.DataFrame([
    ['Yes',  'No', 'No'],
    ['No',  'Yes', 'No'],
    ['No',  'Yes', 'Yes'],
  ],
  columns=['Fever', 'Cough', 'Respiratory Problems']
)
test_examples

,Fever,Cough,Respiratory Problems
0,Yes,No,No
1,No,Yes,No
2,No,Yes,Yes


In [58]:
# Cargar los datos de COVID-19
data = pd.read_csv(path_covid19, index_col=0)
data

,Fever,Cough,Respiratory Problems,Infected
Id,,,,
1,No,No,No,No
2,Yes,Yes,Yes,Yes
3,Yes,Yes,No,No
4,Yes,No,Yes,Yes
5,Yes,Yes,Yes,Yes
6,No,Yes,No,No
7,Yes,No,Yes,Yes
8,Yes,No,Yes,Yes
9,No,Yes,Yes,Yes


Al igual que en el ejercicio anterior para usar la distancia de `euclídea` necesitamos trasformar las etiquetas de las clases en un formato numérico. Para ello podemos usar la conversión llamada `one-hot` que consiste en transformar cada columna que contenga una clase en tantas columnas como sus categorías.

El paquete `Pandas` contiene una función que realiza esta transformación, `pd.get_dummies()` y que podemos usar, o bien, se puede transformar implementando una nueva función que lo haga.

In [65]:
# TODO: Transformación de los datos de entrenamiento (data) a un formato one-hot
data_encoded = pd.get_dummies(data)
data_encoded

,Fever_No,Fever_Yes,Cough_No,Cough_Yes,Respiratory Problems_No,Respiratory Problems_Yes,Infected_No,Infected_Yes
Id,,,,,,,,
1,1,0,1,0,1,0,1,0
2,0,1,0,1,0,1,0,1
3,0,1,0,1,1,0,1,0
4,0,1,1,0,0,1,0,1
5,0,1,0,1,0,1,0,1
6,1,0,0,1,1,0,1,0
7,0,1,1,0,0,1,0,1
8,0,1,1,0,0,1,0,1
9,1,0,0,1,0,1,0,1


Vamos a usar la misma transformación anterior pero en esta ocasión a los datos de las nuevas muestras, variable `test_examples`.

In [66]:
# TODO: Transformación de los datos de test (test_data) a un formato one-hot
test_examples_encoded = pd.get_dummies(test_examples)

A continuación buscaremos los k vecinos más cercanos a las nuevas muestras del conjunto de entrenamiento y presentaremos la predicción de su clase.

In [68]:
# Ejemplo de clasificación para COVID-19tenis con k=3 de las muestras de test (test_data) respecto de las de entrenamiento (data)
# Ejemplo de clasificación para tenis con k=3 de las muestras de test (test_data) respecto de las de entrenamiento (data)
# Preparamos los datos de entrenamiento
X_train = data_encoded.drop(['Infected_Yes','Infected_No'], axis=1)
training_vector = X_train.values
training_labels = data_encoded['Infected_Yes'].values


# Agregamos las columnas faltantes al DataFrame de test_examples_encoded que están en X_train pero no en test_examples_encoded
missing_cols = set(X_train.columns) - set(test_examples_encoded.columns)
for c in missing_cols:
    test_examples_encoded[c] = 0

# Asegúrese de que el orden de las columnas coincida
test_examples_encoded = test_examples_encoded[X_train.columns]

# Convertimos el dataframe procesado en un array de numpy para usarlo en la clasificación
test_vectors = test_examples_encoded.values
print(test_vectors)

k = 3
for index, test_vec in test_examples_encoded.iterrows():
    test_vec = test_vec.values
    result = k_nearest_neighbors(training_vector, training_labels, test_vec, k)
    print(f"La muestra {index} pertenece a la clase: {'Yes' if result == 1 else 'No'}")

[[0 1 1 0 1 0]
 [1 0 0 1 1 0]
 [1 0 0 1 0 1]]
Etiquetas más cercanas [0, 0, 0]
La muestra 0 pertenece a la clase: No
Etiquetas más cercanas [0, 0, 0]
La muestra 1 pertenece a la clase: No
Etiquetas más cercanas [0, 1, 1]
La muestra 2 pertenece a la clase: Yes


## Implementación con scikit-learn

El paquete `scikit-learn` contiene diferentes funciones basadas en algoritmos de vecindad. Para clasificación existe la función `sklearn.neighbors.KNeighborsClassifier()` que podemos usar para entrenar en primer lugar y predecir nuevos valores a continuación.

In [69]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3) # La distancia euclídea se usa por defecto


X_train = data_encoded.drop(['Infected_Yes','Infected_No'], axis=1)
y_train = data_encoded['Infected_Yes'].values


# Clasificación de 'test_data' respecto a 'data' usando sklearn
model.fit(X_train, y_train)

# Transformamos los datos de test (test_examples) a un formato one-hot
test_examples_encoded = pd.get_dummies(test_examples)

# Agregamos las columnas faltantes al DataFrame de test_examples_encoded que están en X_train pero no en test_examples_encoded
missing_cols = set(X_train.columns) - set(test_examples_encoded.columns)
for c in missing_cols:
    test_examples_encoded[c] = 0

# Aseguramos que el orden de las columnas en los datos de prueba coincida con los datos de entrenamiento
test_examples_encoded = test_examples_encoded.reindex(columns=X_train.columns, fill_value=0)

# Realizamos la clasificación de las muestras de prueba
predictions = model.predict(test_examples_encoded)

print(predictions)

[0 0 1]
